In [1]:
import sys
sys.path.append("..")


from pytket import OpType, Circuit
from canopus.utils import tket_to_qiskit
from canopus.synthesis import rebase_to_tk2
import regulus
from pytket.circuit.display import render_circuit_jupyter
import pytket.qasm
import pytket.passes
from math import pi

Commutative CNOTs

In [8]:
circ = Circuit(5)
circ.CX(0, 1)
circ.CX(3, 2)
circ.CX(2, 1)
circ.CX(3, 4)
tket_to_qiskit(circ).draw()

q_0: ──■───────
     ┌─┴─┐┌───┐
q_1: ┤ X ├┤ X ├
     ├───┤└─┬─┘
q_2: ┤ X ├──■──
     └─┬─┘     
q_3: ──■────■──
          ┌─┴─┐
q_4: ─────┤ X ├
          └───┘

In [9]:
circ_can = rebase_to_tk2(circ)
# render_circuit_jupyter(circ_can)
tket_to_qiskit(circ_can).draw(fold=1000)

┌────────────────┐  ┌───────────────┐ ┌─────────────┐                                       
q_0: ──┤ U3(7π/2,π,π/2) ├──┤0              ├─┤ U3(π/2,0,π) ├───────────────────────────────────────
      ┌┴────────────────┤  │  Tk2(π/2,0,0) │ └─────────────┘  ┌───────────────┐ ┌─────────────────┐
q_1: ─┤ U3(π,-π/2,7π/2) ├──┤1              ├──────────────────┤1              ├─┤ U3(0,-π/2,3π/2) ├
     ┌┴─────────────────┴─┐├───────────────┤┌───────────────┐ │  Tk2(π/2,0,0) │ └─┬─────────────┬─┘
q_2: ┤ U3(3π/2,-π/2,3π/2) ├┤1              ├┤ U3(π/2,0,π/2) ├─┤0              ├───┤ U3(π/2,0,π) ├──
     └──┬──────────────┬──┘│  Tk2(π/2,0,0) │├───────────────┤ └┬─────────────┬┘   └─────────────┘  
q_3: ───┤ U3(π/2,0,4π) ├───┤0              ├┤0              ├──┤ U3(π/2,0,π) ├─────────────────────
     ┌──┴──────────────┴──┐└───────────────┘│  Tk2(π/2,0,0) │┌─┴─────────────┴─┐                   
q_4: ┤ U3(3π/2,-π/2,3π/2) ├─────────────────┤1              ├┤ U3(0,-π/2,3π/2) ├───────────────────
     └────────────────────┘                 └───────────────┘└─────────────────┘

In [4]:
c = regulus.Circuit.from_tket(circ)
# ([regulus.gates.X.on(1, 0), regulus.gates.X.on(1, 2)])
c = regulus.transforms.rebase.rebase_to_canonical(c)
c.to_qiskit().draw(fold=200)

┌─────────────────┐ ┌───────────────┐┌─────────────────┐                                     
q_0: ─┤ U3(π/2,π,-3π/4) ├─┤0              ├┤ U3(π/2,-3π/4,0) ├─────────────────────────────────────
     ┌┴─────────────────┴┐│  Can(π/2,0,0) │├─────────────────┴┐┌───────────────┐┌─────────────────┐
q_1: ┤ U3(π/4,-π/2,-π/2) ├┤1              ├┤ U3(π/2,-π/2,π/2) ├┤0              ├┤ U3(π/4,π/2,π/2) ├
     └┬─────────────────┬┘├───────────────┤├──────────────────┤│  Can(π/2,0,0) │├─────────────────┤
q_2: ─┤ U3(π/2,π,-3π/4) ├─┤0              ├┤ U3(π/2,-π/2,π/2) ├┤1              ├┤ U3(π/2,-3π/4,0) ├
     ┌┴─────────────────┴┐│  Can(π/2,0,0) │├──────────────────┤├───────────────┤├─────────────────┤
q_3: ┤ U3(π/4,-π/2,-π/2) ├┤1              ├┤ U3(π/2,-π/2,π/2) ├┤0              ├┤ U3(π/4,π/2,π/2) ├
     └┬─────────────────┬┘└───────────────┘└──────────────────┘│  Can(π/2,0,0) │├─────────────────┤
q_4: ─┤ U3(π/2,π,-3π/4) ├──────────────────────────────────────┤1              ├┤ U3(π/2,-3π/4,0) ├
      └─────────────────┘                                      └───────────────┘└─────────────────┘

Non-Commutative CNOTs

In [5]:
circ = Circuit(4)
circ.CX(0, 1)
circ.CX(1, 2)
# circ.CX(2, 1)
# circ.CX(4, 3)
tket_to_qiskit(circ).draw()

q_0: ──■───────
     ┌─┴─┐     
q_1: ┤ X ├──■──
     └───┘┌─┴─┐
q_2: ─────┤ X ├
          └───┘
q_3: ──────────

In [6]:
circ_can = rebase_to_tk2(circ)
tket_to_qiskit(circ_can).draw(fold=1000)

┌────────────────┐  ┌───────────────┐ ┌─────────────┐                                     
q_0: ──┤ U3(7π/2,π,π/2) ├──┤0              ├─┤ U3(π/2,0,π) ├─────────────────────────────────────
     ┌─┴────────────────┴─┐│  Tk2(π/2,0,0) │┌┴─────────────┴┐┌───────────────┐  ┌─────────────┐  
q_1: ┤ U3(3π/2,-π/2,3π/2) ├┤1              ├┤ U3(π/2,0,π/2) ├┤0              ├──┤ U3(π/2,0,π) ├──
     ├────────────────────┤└───────────────┘└───────────────┘│  Tk2(π/2,0,0) │┌─┴─────────────┴─┐
q_2: ┤ U3(3π/2,-π/2,3π/2) ├──────────────────────────────────┤1              ├┤ U3(0,-π/2,3π/2) ├
     └────────────────────┘                                  └───────────────┘└─────────────────┘
q_3: ────────────────────────────────────────────────────────────────────────────────────────────